## Time-series Momentum strategy

In [3]:
def getPNL_TimeSeriesMomentum(dates, GenericPrice, mom_period):
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    ''' This function implements the time-series momentum startegy for the construction of a portfolio.  '''
    ''' As the name says, the momentum is used as signal to define the future position on a derivative.  '''
    ''' The output is the pnl of the portfolio whose evolution can be graphically investigated using     '''
    ''' the command plt.plot(np.cumprod(1+pnl)). As for the input, dates are the date which has been     '''
    ''' transformed into dateime object; GenericPrice is the dataset of the raw prices, mom_period       '''
    ''' refers to the window for the momentum (N.B. mom_period should be lower than 260)                 '''
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    expo = GenericPrice.copy() * 0.0
    name_asset = GenericPrice.columns
    pnl = np.full(GenericPrice.shape[0], 0.0)
    
    for t in range(261, GenericPrice.shape[0]-1):
        for i in range(len(name_asset)):
            mom_i = GenericPrice.iloc[t, i]/ GenericPrice.iloc[t-mom_period, i] - 1
            vol_i = np.std(np.array(GenericPrice.iloc[(t-260):t, i])/np.array(GenericPrice.iloc[(t-261):(t-1), i]) - 1.0) * np.sqrt(260)
            ## here we allocate based on a target volatility of the single future of 15%
            if mom_i > 0:
                expo.iloc[t+1, i] = 1.0 * (0.15 / vol_i)
            else:
                expo.iloc[t+1, i] = -1.0 * (0.15 / vol_i)
    
    genericPerf = GenericPrice / GenericPrice.shift(+1) - 1
    genericPerf.fillna(0.0, inplace=True)
    
    for i in range(262, genericPerf.shape[0]):
        ## Here we adjust to have a target exante volatility for the all portfolio of 15%
        window = genericPerf.iloc[(i-260):i, : ]
        
        cov_temp = np.cov(window.T) * 260
        ex_ante_vol = np.sqrt(np.dot(np.dot(expo.iloc[i, : ], cov_temp), expo.iloc[i, : ].T)) 
        
        lev_temp = 0.15 / ex_ante_vol
        expo.iloc[i, : ] = expo.iloc[i, : ] * lev_temp
        
        pnl[i] = np.dot(genericPerf.iloc[i, :], expo.iloc[i,:].T)
    
    return pnl

## Cross-Sectional Momentum Strategy

In [4]:
def getPNL_CrossSectionalMomentum(dates, GenericPrice, mom_period):
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    ''' This function implements the cross-sectional momentum startegy for the construction of a         '''
    ''' portfolio. The signal is given by the momentum relative performance and is used to derive the    '''
    ''' future position on a derivative. The output is the pnl of the portfolio whose evolution can be   '''
    ''' graphically investigated usingthe command plt.plot(np.cumprod(1+pnl)). As for the input, dates   '''
    ''' are the date which has been transformed into dateime object; GenericPrice is the dataset of the  '''
    ''' raw prices, mom_period refers to the window for the momentum (N.B. mom_period should be          '''
    ''' lower than 260).                                                                                 '''
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    expo = GenericPrice.copy() * 0.0
    name_asset = GenericPrice.columns
    pnl = np.full(GenericPrice.shape[0], 0.0)
    
    for t in range(261, GenericPrice.shape[0]-1):
        mom_temp = GenericPrice.iloc[t, : ]/ GenericPrice.iloc[t-mom_period, : ] - 1
        mom_temp_avg = np.mean(mom_temp)
        
        for i in range(len(name_asset)):
            vol_i = np.std(np.array(GenericPrice.iloc[(t-260):t, i])/np.array(GenericPrice.iloc[(t-261):(t-1), i]) - 1.0) * np.sqrt(260)
            if mom_temp[i] - mom_temp_avg > 0:
                expo.iloc[t+1, i] = 1.0 * (0.15 / vol_i)
            else:
                expo.iloc[t+1, i] = -1.0 * (0.15 / vol_i)
    
    genericPerf = GenericPrice / GenericPrice.shift(+1) - 1
    genericPerf.fillna(0.0, inplace=True)
    
    for i in range(262, genericPerf.shape[0]):
        window = genericPerf.iloc[(i-260):i, : ]
        
        cov_temp = np.cov(window.T) * 260
        ex_ante_vol = np.sqrt(np.dot(np.dot(expo.iloc[i, : ], cov_temp), expo.iloc[i, : ].T)) 
        
        lev_temp = 0.15 / ex_ante_vol
        expo.iloc[i, : ] = expo.iloc[i, : ] * lev_temp
        
        pnl[i] = np.dot(genericPerf.iloc[i, :], expo.iloc[i,:].T)
    
    return pnl

## Carry Signal Strategy

In [5]:
#(P_t - P_t+1)/P_t+1

def getPNL_CarryStrategy(dates, CarrySignals, GenericPrice):
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    ''' This function implements the carry signal startegy for the construction of a portfolio.          '''
    ''' The carry signal is computed as the percentage difference between the two most liquid contracts. '''
    ''' In this specific case, the input CarrySignal correspond to a dataset whose carry signals have    '''
    ''' already been computed. A positive carry signal is associated to a term structure backwardation   '''
    ''' shaped. Assuming no future shifts in the TS the prices are expected to increase. In this         '''
    ''' situation the function goes long on derivatives associated to a positive carry signal. The       '''
    ''' output is the pnl of the portfolio whose evolution can be graphically investigated using the     '''
    '''  command plt.plot(np.cumprod(1+pnl)). As for the input, dates are the date which has been        '''
    ''' transformed into dateime object; GenericPrice is the dataset of the raw prices.                  '''
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    expo = GenericPrice.copy() * 0.0
    name_assets = CarrySignals.columns
    pnl = np.full(GenericPrice.shape[0], 0.0)
    
    for t in range(261, GenericPrice.shape[0]-1):
        for i in range(len(name_assets)):
            vol_i = np.std(np.array(GenericPrice.iloc[(t-260):t, i])/np.array(GenericPrice.iloc[(t-261):(t-1), i]) - 1.0) * np.sqrt(260)
            if CarrySignals.iloc[t, i] > 0: ## I could take some MA for the carry signals
                expo.iloc[t+1, i] = 1.0 * (0.15/vol_i)
            else:
                expo.iloc[t+1, i] = -1.0 * (0.15/vol_i)

    genericPerf = GenericPrice / GenericPrice.shift(+1) - 1
    genericPerf.fillna(0.0, inplace=True)
    
    for i in range(262, genericPerf.shape[0]):
        window = genericPerf.iloc[(i-260):i, : ]
        
        cov_temp = np.cov(window.T) * 260
        ex_ante_vol = np.sqrt(np.dot(np.dot(expo.iloc[i, : ], cov_temp), expo.iloc[i, : ].T)) 
        
        lev_temp = 0.15 / ex_ante_vol
        expo.iloc[i, : ] = expo.iloc[i, : ] * lev_temp
        
        pnl[i] = np.dot(genericPerf.iloc[i, :], expo.iloc[i,:].T)

    return pnl